In [34]:
import warnings
warnings.filterwarnings('ignore')
import json
from tensorflow import keras
from transformers import AutoTokenizer

from modules.utils import *
from modules.dataframe_tools import *
from keras.models import load_model

In [25]:
path = "./dataset"
with open(path+'/training_set.json') as f:
    raw_train_data = json.load(f)



In [26]:
def explore_level(level, key, tab=0):
  # general information
  print(f'{"  " * tab}{tab}. {key}', end=' ')

  # type related information
  if type(level) == list:
    print("")
    level = level[0]
  elif type(level) == dict:
    print("")
  else:
    print("")
    return
  
  # if not returned explores next level
  for key, value in level.items():
    explore_level(value, key, tab + 1)


explore_level(raw_train_data, 'dataset')

0. dataset 
  1. data 
    2. title 
    2. paragraphs 
      3. context 
      3. qas 
        4. answers 
          5. answer_start 
          5. text 
        4. question 
        4. id 
  1. version 


In [27]:
df = create_df_train(path+'/training_set.json', [])
errors_id = []

## Wrong questions
Here we try to find questions that could be wrong. We find:
- question that recite  "I couldn't could up with another question. But i need to fill this space because I can't submit the hit. ". This is obviusly a wrong question and needs to be addressed.
- some repeated question. These could be wrong and need more in depth analysis.
- some meaningless question with less than 4 characters.

In [28]:
possibly_wrong = [q for q in df['question'] if '?' not in q]
possibly_wrong

['What song did Beyoncé perform at the first inaugural dance for the Obamas.',
 'Beyoncé was one of how many women on Billboard magazine\'s 2011 "Top 20 Hot 100 Songwriters" list.',
 'What word is often used to describe Beyonce/',
 'What is the state fossil',
 'How long have railroads been important since in Montana',
 'Between what years did the Native population increase by 27.9%',
 'Who promoted settlement in Montana in the early 1900s',
 "How many new classes of antibacterial antibiotics was introduced in the late 2000's/",
 'When does the spread of antibacterial resistance frequently occurs/',
 'Name four sound profiles that would result in bass distortion on pre-2007 iPods.',
 "Which Formula One racing team developed the C-X75's used for filming.",
 'How many day did filming take',
 'How many schools collapsed in Mianyang City',
 'What catastrophe inspired them to make a building design code/ ',
 'What Red Cross team left Taipei on May 16',
 'Growth of solar water heating develop

In [29]:
# Find duplicated questions in the dataset
dr = df[df["question"].duplicated()]
# Look if this question have duplicated answers
dr = dr[dr["answer_text"].duplicated()]
# Add the ids of duplicated elements to error list
errors_id += dr["id"].to_list()

In [30]:
errors_id += df[df["question"] == "I couldn't could up with another question. But i need to fill this space because I can't submit the hit. "]["id"].to_list()
errors_id += df[[len(q) < 5 for q in df["question"].to_list()]]["id"].to_list()

In [31]:
df[df["id"].isin(errors_id)][["question", "answer_text"]].shape

(33, 2)

In [32]:
with open("errors.txt", "w") as out:
  for a in errors_id:
    out.write("{}\n".format(a))

## Counting the final element in the various sets
With the list of errors completed we can find how many element we have in each set (Train, Validation and Test)

In [36]:
path_to_json = path+'/training_set.json'
test_size = 2
df = create_df_train(path_to_json=path_to_json, errors=errors_id, parag_dump=test_size)
test = create_df_test(path_to_json, errors_id, parag_lift=test_size)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

df = process_dataset(df, tokenizer, answer_available=True)
x_train,y_train, x_val, y_val = train_test_split_on_title(df)




Token indices sequence length is longer than the specified maximum sequence length for this model (720 > 512). Running this sequence through the model will result in indexing errors


In [39]:
print(f"Train set size = \t\t{len(x_train[0])}")
print(f"Validation set size = \t\t{len(x_val[0])}")
print(f"Test set size = \t\t{test.shape[0]}")


Train set size = 		64812
Validation set size = 		21719
Test set size = 		1027
